In [ ]:
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)
import pandas as pd
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [1]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

In [8]:
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

# Generate samples
param_values = saltelli.sample(problem, 1000)

# Run model (example)
Y = Ishigami.evaluate(param_values)

# # Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# # Print the first-order sensitivity indices
print(Si['S1'])

Parameter S1 S1_conf ST ST_conf
x1 0.307975 0.066360 0.560137 0.078083
x2 0.447767 0.060502 0.438722 0.041146
x3 -0.004255 0.067962 0.242845 0.027428

Parameter_1 Parameter_2 S2 S2_conf
x1 x2 0.012205 0.078062
x1 x3 0.251526 0.112773
x2 x3 -0.009954 0.064969
[ 0.30797549  0.44776661 -0.00425452]


In [ ]:
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df
df2=df.copy()